In [227]:
import ncbi_genome_download as ngd
import os, re, gzip
from ete3 import NCBITaxa
import os

In [228]:
ncbi = NCBITaxa()
workpath = os.path.join("../" + "NCBITaxa/")
controlpath = os.path.join("./example_control")
try:
    os.mkdir(workpath)
except FileExistsError:
    print("File exists: " + workpath)
print("Work Path:" + workpath)
print("Control datasets path: " + controlpath)

Work Path:../NCBITaxa/
Control datasets path: ./example_control


In [239]:
with open(controlpath, "r") as f:
    l = f.readlines()

output = [[]]

for x in l:
    output[-1].append(x)
    if x == '\n':
        output.append([])
output[0].remove('\n')
positive = []
negative = []
whole = []
group = output[0][0]
for i in output[1]:
    positive.append(i[:-1])
    whole.append(i[:-1])
for i in output[2]:
    negative.append(i[:-1])
    whole.append(i[:-1])
print("####################################################################################")
print("Taxon group: " + group)
print("Positve control group:")
for i in positive:
    print(i)
print()
print("Negative control group:")
for i in negative:
    print(i)


####################################################################################
Taxon group: bacteria

Positve control group:
Mycobacterium tuberculosis


Negative control group:
Mycobacterium bovis
Mycobacterium avium
Mycobacterium smegmatis
Mycobacterium abscessus
Mycobacterium kansasii
Mycobacterium fortuitum


In [230]:
def getTaxid(namelist):
    # Get Taxon id
    accessid = []
    for i in namelist:
        name2taxid = ncbi.get_name_translator([i])
        if name2taxid == {}:
            print("Wrong Taxon name!")
            return
        else:
            accessid.append(name2taxid)
    return accessid

In [231]:
def ungz_all_fasta(top_dir):
    if os.path.isdir(top_dir):
        if os.listdir(top_dir) == []:
            return
        for i in os.listdir(top_dir):
            if i.split(".")[-2:] == ["fna", "gz"]:
                os.system("gunzip " + top_dir + "/" + i)
            get_all_fasta(top_dir + "/" + i)
    else:
        return
def rm_not_fasta(current_dir):
    remove_list = []
    for i in os.listdir(current_dir):
        if i[-4:] != ".fna":
            remove_list.append(i)
    for i in remove_list:
        os.system("rm -r " + current_dir + i)

In [236]:
def download_db(taxnamelist, group):
    print("Group: " + group)
    for i in getTaxid(taxnamelist):
        Taxon = list(i.keys())[0]
        Taxonid = str(list(i.values())[0][0])
        outdir = workpath + "_".join(Taxon.split(" ")) + "/"
        try:
            os.mkdir(outdir)
        except FileExistsError:
            print("Path exists: "+ outdir)
        print("####################################################################################")
        print("Downloading complete sequence in fasta from NCBI database...\n" + 
            "Taxon: " + Taxon + "\n" + 
            "Taxon id: " + Taxonid + "\n" + 
            "Directory: " + outdir)
        print("Executing: " + "ncbi-genome-download -t " + Taxonid +  " -F fasta --assembly-level complete "  +" -o " + outdir + " " + group)
        os.system("ncbi-genome-download -t " + Taxonid +  " -F fasta --assembly-level complete "  +" -o " + outdir + " " + group)
        print("...Taxon " + Taxon + " downloaded complete!")
        print("Unzip and re-organizing...")
        ungz_all_fasta(outdir)
    for i in os.walk(outdir):
        for j in i[2]:
            if j[-4:] == ".fna":
                os.system("cp " + i[0]+"/"+j + " " + outdir)
    rm_not_fasta(outdir)

In [234]:
download_db(whole, "bacteria")

Group: bacteria
####################################################################################
Taxon: Mycobacterium tuberculosis
Taxon id: 1773
Directory: ../NCBITaxa/Mycobacterium_tuberculosis/
#Excuting: ncbi-genome-download -T 1773 -F fasta --assembly-level complete  -o ../NCBITaxa/Mycobacterium_tuberculosis/ bacteria
...Taxon Mycobacterium tuberculosis downloaded complete!
####################################################################################
Taxon: Mycobacterium bovis
Taxon id: 1765
Directory: ../NCBITaxa/Mycobacterium_bovis/
#Excuting: ncbi-genome-download -T 1765 -F fasta --assembly-level complete  -o ../NCBITaxa/Mycobacterium_bovis/ bacteria
...Taxon Mycobacterium bovis downloaded complete!
####################################################################################
Taxon: Mycobacterium avium
Taxon id: 1764
Directory: ../NCBITaxa/Mycobacterium_avium/
#Excuting: ncbi-genome-download -T 1764 -F fasta --assembly-level complete  -o ../NCBITaxa/Mycobacter